In [1]:
import pandas as pd
import numpy as mp
import torch
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW, DistilBertTokenizer, DistilBertForSequenceClassification, BertTokenizer, BertForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

2.6.0+cu124
12.4
True


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
# Modeli yükleme
#model = DistilBertForSequenceClassification.from_pretrained('./uclumodelm6')
model = BertForSequenceClassification.from_pretrained('./modelb')
#model = RobertaForSequenceClassification.from_pretrained('./rrfullazneuthappiness3modelm6')
# Tokenizer'ı yükleme
#mytokenizer = DistilBertTokenizer.from_pretrained('./uclutokenm6')
mytokenizer = BertTokenizer.from_pretrained('./tokenb')
#mytokenizer = RobertaTokenizer.from_pretrained('./rrfullazneuthappiness3tokenm6')

In [5]:
model = model.to(device) # Önemli

In [6]:
def predict_sentence(sentence):
    # Cümleyi tokenize et
    inputs = mytokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # Modeli eval moduna al
    model.eval()
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits
    probs = torch.sigmoid(logits)  # Multi-label olduğu için sigmoid

    
    # Olasılıkları ayarla
    for i in range(probs.size(0)):
        # class indexlerine göre düzeltmeler
        for j, class_name in enumerate(mlb.classes):
            if class_name == "neutral":
                probs[i][j] *= 0.8
            elif class_name == "happiness":
                probs[i][j] *= 0.85

    
    predicted_index = torch.argmax(probs, dim=1).item()  # En yüksek olasılığı al
    
    # Sınıf adlarını çözmek için
    class_names = mlb.classes
    predicted_class_name = class_names[predicted_index]
    
    return predicted_class_name, probs.squeeze().tolist()

In [7]:
def predict_and_display_probabilities(sentence):
    predicted_class_name, probabilities = predict_sentence(sentence)
    
    print(f"\nPredicted Emotion: **{predicted_class_name}**\n")
    
    
    # Sınıf isimleri ile olasılıkları eşleştir
    class_probabilities = list(zip(mlb.classes, probabilities))  
    
    print("Probabilities for each emotion class:")
    
    # Olasılıkları büyükten küçüğe sıralama
    sorted_class_probabilities = sorted(class_probabilities, key=lambda x: x[1], reverse=True)
    # Sınıf olasılıklarını yazdır
    for class_name, prob in sorted_class_probabilities:
        print(f"{class_name}: {prob*100:.2f}%")

In [8]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(classes=['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral'])

In [9]:
sentence = "I remember her sinking beneath the waters, and then, and then... And so it was, that love was the gravest sin of all."

In [10]:
sentence = "I think I will miss you forever, like the stars miss the sun in the morning sky."

In [11]:
sentence = "I really loved her, but she left."

In [12]:
sentence = "I love you my darling."

In [13]:
sentence = "This is both incredible and terrifying…. Just think these are the things they are telling us about. 😮"

In [14]:
sentence = "When he says tissue damage he really means brain damage, you have the possibility of losing more capabilities after procedure..... Yeah this is a pass, thanks"

In [15]:
sentence = "Don't get me wrong, this is a wonderful advancement that could help a lot of people, but at the same time, it feels like the backstory to a dystopian novel."

In [16]:
sentence = "I don't need any thing artificial connected to my brain The body is it's own intelligence."

In [17]:
sentence = "This is getting out of hand, STOP THE CHIPPING, these guys want to control us with these things"

In [18]:
sentence = "The fact that most people in these comments are approving of this type of technology is insane to me. It blows my mind how far tech has overtaken our lives."

In [19]:
sentence = "this is a pass no thanks"

In [20]:
sentence = "This is getting out of hand, STOP THE CHIPPING, these guys want to control us with these things"

In [21]:
sentence = "I'm not gonna use this kind of chip that can control my mind."

In [22]:
predict_and_display_probabilities(sentence)


Predicted Emotion: **anger**

Probabilities for each emotion class:
anger: 91.90%
neutral: 24.73%
fear: 8.78%
surprise: 8.60%
sadness: 5.20%
disgust: 2.61%
happiness: 1.51%


In [23]:
predict_and_display_probabilities(sentence)


Predicted Emotion: **anger**

Probabilities for each emotion class:
anger: 91.90%
neutral: 24.73%
fear: 8.78%
surprise: 8.60%
sadness: 5.20%
disgust: 2.61%
happiness: 1.51%


### Gradio ile arayüz

In [24]:
import gradio as gr

def predict_gradio_interface(sentence):
    predicted_class_name, probabilities = predict_sentence(sentence)
    class_probabilities = list(zip(mlb.classes, probabilities))
    sorted_class_probabilities = sorted(class_probabilities, key=lambda x: x[1], reverse=True)

    result_str = f"**Predicted Emotion:** {predicted_class_name}**\n\n"
    result_str += "### Probabilities for each emotion class:\n\n"
    for class_name, prob in sorted_class_probabilities:
        result_str += f"- {class_name}: {prob*100:.2f}%\n"
    return result_str

interface = gr.Interface(
    fn=predict_gradio_interface,
    inputs=gr.Textbox(lines=3, placeholder="Enter a sentence here..."),
    outputs="markdown",
    allow_flagging="never",
    title="Emotion Detection with BERT",
    description="Type a sentence and see predicted emotion with probability scores."
)

interface.launch(share=True)  # "share=True" ile dış bağlantı da alabilirsin

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://30f5933a4d4e85f3db.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
